In [1]:
%pwd

'/ocean/projects/asc170022p/mtragoza/lung-project'

In [2]:
import sys
sys.path.append('../param_search')
import param_search as ps

In [3]:
# define a job template and name format
template = '''\
#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --account=asc170022p
#SBATCH --partition=GPU-shared
#SBATCH --gres=gpu:1
#SBATCH -x v034
#SBATCH --time=48:00:00
#SBATCH -o %J.stdout
#SBATCH -e %J.stderr
#SBATCH --mail-type=all

hostname
pwd
module load anaconda3
conda activate /ocean/projects/asc170022p/mtragoza/mambaforge/envs/4DCT
nvidia-smi

data_root=../../data/{data_name}
data_root=$(realpath $data_root)

case_name=Case{case_id:d}Pack
case_dir=$data_root/$case_name

image_dir=$case_dir/NIFTI
mask_dir=$case_dir/TotalSegment
output_dir=$case_dir/CorrField

mkdir -p $output_dir

fixed_name=case{case_id:d}_T{fixed_phase:02d}
fixed_image=$image_dir/$fixed_name.nii.gz
fixed_mask=$mask_dir/$fixed_name/lung_combined_mask.nii.gz

moving_name=case{case_id:d}_T{moving_phase:02d}
moving_image=$image_dir/$moving_name.nii.gz

reg_name=case{case_id:d}_T{moving_phase:02d}_T{fixed_phase:02d}
output_path=$output_dir/$reg_name

cd ../../../Lung250M-4B
python -m corrfield -F $fixed_image -M $moving_image -m $fixed_mask -O $output_path

echo Done
'''
name_format = 'register_{data_name}_case{case_id:d}_T{moving_phase:02d}_T{fixed_phase:02d}'

In [7]:
param_space = ps.ParamSpace(
    data_name='Emory-4DCT',
    case_id=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    fixed_phase=[50],
    moving_phase=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
)

for p in param_space:
    print(name_format.format(**p))
    
print(len(param_space))

register_Emory-4DCT_case1_T00_T50
register_Emory-4DCT_case1_T10_T50
register_Emory-4DCT_case1_T20_T50
register_Emory-4DCT_case1_T30_T50
register_Emory-4DCT_case1_T40_T50
register_Emory-4DCT_case1_T50_T50
register_Emory-4DCT_case1_T60_T50
register_Emory-4DCT_case1_T70_T50
register_Emory-4DCT_case1_T80_T50
register_Emory-4DCT_case1_T90_T50
register_Emory-4DCT_case2_T00_T50
register_Emory-4DCT_case2_T10_T50
register_Emory-4DCT_case2_T20_T50
register_Emory-4DCT_case2_T30_T50
register_Emory-4DCT_case2_T40_T50
register_Emory-4DCT_case2_T50_T50
register_Emory-4DCT_case2_T60_T50
register_Emory-4DCT_case2_T70_T50
register_Emory-4DCT_case2_T80_T50
register_Emory-4DCT_case2_T90_T50
register_Emory-4DCT_case3_T00_T50
register_Emory-4DCT_case3_T10_T50
register_Emory-4DCT_case3_T20_T50
register_Emory-4DCT_case3_T30_T50
register_Emory-4DCT_case3_T40_T50
register_Emory-4DCT_case3_T50_T50
register_Emory-4DCT_case3_T60_T50
register_Emory-4DCT_case3_T70_T50
register_Emory-4DCT_case3_T80_T50
register_Emory

In [8]:
jobs = ps.submit(template, name_format, param_space, work_dir='2023-01-15_register')
jobs.to_csv('2023-01-15_register.jobs')
jobs

100%|██████████| 100/100 [00:00<00:00, 124.11it/s]


,data_name,case_id,fixed_phase,moving_phase,job_name,job_id,partition,job_state,node_id,runtime,work_dir,array_idx
0,Emory-4DCT,1,50,0,register_Emory-4DCT_case1_T00_T50,21681668,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
1,Emory-4DCT,1,50,10,register_Emory-4DCT_case1_T10_T50,21681669,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
2,Emory-4DCT,1,50,20,register_Emory-4DCT_case1_T20_T50,21681670,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
3,Emory-4DCT,1,50,30,register_Emory-4DCT_case1_T30_T50,21681671,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
4,Emory-4DCT,1,50,40,register_Emory-4DCT_case1_T40_T50,21681672,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Emory-4DCT,10,50,50,register_Emory-4DCT_case10_T50_T50,21681763,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
96,Emory-4DCT,10,50,60,register_Emory-4DCT_case10_T60_T50,21681764,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
97,Emory-4DCT,10,50,70,register_Emory-4DCT_case10_T70_T50,21681765,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
98,Emory-4DCT,10,50,80,register_Emory-4DCT_case10_T80_T50,21681766,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN


In [9]:
import pandas as pd
jobs = pd.read_csv('2023-01-15_register.jobs', index_col=0)
status = ps.status(jobs)
status

,index,data_name,case_id,fixed_phase,moving_phase,job_name,partition,job_state,node_id,runtime,work_dir,array_idx,stdout,stderr
job_id,,,,,,,,,,,,,,
21681668,0,Emory-4DCT,1,50,0,register_Emory-4DCT_case1_T00_T50,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
21681669,1,Emory-4DCT,1,50,10,register_Emory-4DCT_case1_T10_T50,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
21681670,2,Emory-4DCT,1,50,20,register_Emory-4DCT_case1_T20_T50,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
21681671,3,Emory-4DCT,1,50,30,register_Emory-4DCT_case1_T30_T50,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
21681672,4,Emory-4DCT,1,50,40,register_Emory-4DCT_case1_T40_T50,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21681763,95,Emory-4DCT,10,50,50,register_Emory-4DCT_case10_T50_T50,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
21681764,96,Emory-4DCT,10,50,60,register_Emory-4DCT_case10_T60_T50,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
21681765,97,Emory-4DCT,10,50,70,register_Emory-4DCT_case10_T70_T50,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN


In [ ]:
print(status.iloc[0].stderr)

In [18]:
import numpy as np
import xarray as xr
import torch
import nibabel as nib

disp_file = 'data/Emory-4DCT/Case1Pack/CorrField/case1_T00_T50.nii.gz'
disp = torch.from_numpy(
    nib.load(disp_file).get_fdata().astype(np.float32)
)
disp.shape

torch.Size([1, 256, 256, 94, 3])

In [25]:
import data

ds = data.Emory4DCTDataset(data_root='data/Emory-4DCT', case_names=['Case1Pack'])
ds.load_images()
ds.load_masks()
ds.load_disps()

Loading data/Emory-4DCT/Case1Pack/Images/case1_T00_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T10_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T20_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T30_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T40_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T50_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T60_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T70_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T80_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T90_s.img


AttributeError: 'Emory4DCTDataset' object has no attribute 'load_masks'